In [27]:
import pandas as pd
import plotly.express as px

covid = pd.read_csv('full_data.csv')
covid = covid[['date', 'location', 'total_cases', 'total_deaths']]

countries_geo = pd.read_csv('countries.csv')
countries_geo = countries_geo[['latitude', 'longitude', 'name']]
countries_geo.columns = ['latitude', 'longitude', 'location']

In [28]:
covid.head()

,date,location,total_cases,total_deaths
0,2019-12-31,Afghanistan,NaN,NaN
1,2020-01-01,Afghanistan,NaN,NaN
2,2020-01-02,Afghanistan,NaN,NaN
3,2020-01-03,Afghanistan,NaN,NaN
4,2020-01-04,Afghanistan,NaN,NaN


In [29]:
countries_geo.head()

,latitude,longitude,location
0,42.546245,1.601554,Andorra
1,23.424076,53.847818,United Arab Emirates
2,33.939110,67.709953,Afghanistan
3,17.060816,-61.796428,Antigua and Barbuda
4,18.220554,-63.068615,Anguilla


In [30]:
countries = pd.merge(covid, countries_geo, how='inner', on='location')

countries['date'] = pd.to_datetime(countries['date'])
countries = countries[countries.date.dt.year == 2020].copy()
countries.fillna(0, inplace=True)
countries.head()

,date,location,total_cases,total_deaths,latitude,longitude
1,2020-01-01,Afghanistan,0.0,0.0,33.93911,67.709953
2,2020-01-02,Afghanistan,0.0,0.0,33.93911,67.709953
3,2020-01-03,Afghanistan,0.0,0.0,33.93911,67.709953
4,2020-01-04,Afghanistan,0.0,0.0,33.93911,67.709953
5,2020-01-05,Afghanistan,0.0,0.0,33.93911,67.709953


In [31]:
countries_week = countries[countries.date.dt.weekday == 6].copy()
countries_week['week'] = countries_week['date'].dt.isocalendar()['week']
countries_week.head()

,date,location,total_cases,total_deaths,latitude,longitude,week
5,2020-01-05,Afghanistan,0.0,0.0,33.93911,67.709953,1
12,2020-01-12,Afghanistan,0.0,0.0,33.93911,67.709953,2
19,2020-01-19,Afghanistan,0.0,0.0,33.93911,67.709953,3
26,2020-01-26,Afghanistan,0.0,0.0,33.93911,67.709953,4
33,2020-02-02,Afghanistan,0.0,0.0,33.93911,67.709953,5


In [32]:
fig_map = px.scatter_geo(
    countries_week,
    lat = 'latitude',
    lon = 'longitude',
    size = 'total_cases',
    hover_name = 'location',
    hover_data = ["total_cases", "total_deaths"],
    title = "COVID-19 Evolution in 2020",
    animation_frame = "week",
    color="total_cases",
    labels={
        'total_cases': 'Cases',
        'total_deaths': 'Deaths',
        'week': 'Week',
        'latitude': 'Latitude',
        'longitude': 'Longitude'
    },
    projection = 'natural earth'
)

fig_map.show()

In [33]:
top10 = countries_week[countries_week.week == countries_week.week.max()].sort_values('total_cases', ascending=False).iloc[:10, 1].to_list()

fig_line = px.line(
    countries_week[countries_week.location.isin(top10)],
    x="week",
    y="total_cases",
    log_y = False,
    color='location',
    hover_name = 'location',
    hover_data = ["total_cases", "total_deaths"],
    title = "COVID-19 Evolution in 2020 (Top 10 Countries)",
    labels= {
        'total_cases': 'Cases',
        'total_deaths': 'Deaths',
        'week': 'Week',
        'latitude': 'Latitude',
        'longitude': 'Longitude',
        'location': 'Country'
    }
)

fig_line.show()

In [ ]:
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash-table

In [35]:
import os.path
import sys, json
import requests
import subprocess

import numpy as np
import pandas as pd
import plotly.express as px

from requests.exceptions import RequestException
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

from collections import namedtuple

In [36]:
def download_ngrok():
    if not os.path.isfile('ngrok'):
        !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
        !unzip -o ngrok-stable-linux-amd64.zip
    pass

In [37]:
Response = namedtuple('Response', ['url', 'error'])

def get_tunnel():
    try:
        Tunnel = subprocess.Popen(['./ngrok','http','8050'])

        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)

        res = session.get('http://localhost:4040/api/tunnels')
        res.raise_for_status()

        tunnel_str = res.text
        tunnel_cfg = json.loads(tunnel_str)
        tunnel_url = tunnel_cfg['tunnels'][0]['public_url']

        return Response(url=tunnel_url, error=None)
    except RequestException as e:
        return Response(url=None, error=str(e))

In [41]:
%%writefile my_dash_app.py
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

covid = pd.read_csv('full_data.csv')
covid = covid[['date', 'location', 'total_cases', 'total_deaths']]

countries_geo = pd.read_csv('countries.csv')
countries_geo = countries_geo[['latitude', 'longitude', 'name']]
countries_geo.columns = ['latitude', 'longitude', 'location']

countries = pd.merge(covid, countries_geo, how='inner', on='location')

countries['date'] = pd.to_datetime(countries['date'])
countries = countries[countries.date.dt.year == 2020].copy()
countries.fillna(0, inplace=True)

countries_week = countries[countries.date.dt.weekday == 6].copy()
countries_week['week'] = countries_week['date'].dt.isocalendar()['week']

fig_map = px.scatter_geo(
    countries_week,
    lat = 'latitude',
    lon = 'longitude',
    size = 'total_cases',
    hover_name = 'location',
    hover_data = ["total_cases", "total_deaths"],
    title = "COVID-19 Evolution in 2020",
    animation_frame = "week",
    color="total_cases",
    labels={
        'total_cases': 'Cases',
        'total_deaths': 'Deaths',
        'week': 'Week',
        'latitude': 'Latitude',
        'longitude': 'Longitude'
    },
    projection = 'natural earth'
)

top10 = countries_week[countries_week.week == countries_week.week.max()].sort_values('total_cases', ascending=False).iloc[:10, 1].to_list()

fig_line = px.line(
    countries_week[countries_week.location.isin(top10)],
    x="week",
    y="total_cases",
    log_y = False,
    color='location',
    hover_name = 'location',
    hover_data = ["total_cases", "total_deaths"],
    title = "COVID-19 Evolution in 2020 (Top 10 Countries)",
    labels= {
        'total_cases': 'Cases',
        'total_deaths': 'Deaths',
        'week': 'Week',
        'latitude': 'Latitude',
        'longitude': 'Longitude',
        'location': 'Country'
    }
)

app.layout = html.Div(children=[
    html.Div([
        html.H1(children='COVID-19 Evolution in 2020'),

        dcc.Graph(
            id='graph1',
            figure=fig_map
        ),  
    ]),
    html.Div([
        dcc.Graph(
            id='graph2',
            figure=fig_line
        ),  
    ]),
])

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=True)

Overwriting my_dash_app.py


In [39]:
download_ngrok()

In [42]:
tunnel = get_tunnel()
print(tunnel)
!python my_dash_app.py

Response(url='https://a2527c3f2773.ngrok.io', error=None)
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "my_dash_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
